In [99]:
#from pyefd import elliptic_fourier_descriptors
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist
import cv2
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize

from scipy.spatial import KDTree

import sys
sys.path.append("..")

from utils import rasterize

In [52]:
dfs = [[] for _ in range(10)]
for i in range(10):
    for k in range(100):
        df = pd.read_csv(f"../training_data/stroke_{i}_{k+1:04}.csv", header=None, usecols=[0, 1])
        df = (df - df.mean()) / df.std()
        # df = (df-df.min())/(df.max()-df.min())
        dfs[i].append(df)

In [55]:
def rasterize_number(df, size, thickness=5):
    tmp = np.zeros((size, size))
    pts = df[[0,1]] * size
    pts = df[[0,1]] * size/6 + size/2
    pts = pts.to_numpy().astype(np.int32)
    for j in range(len(pts)-1):
        tmp = cv2.line(tmp, pts[j], pts[j+1], 1, thickness)
    tmp  = cv2.flip(tmp, 0)
    return tmp

In [71]:
SIZE = 100
template_size = (SIZE, SIZE)
templates = [np.zeros(template_size) for _ in range(10)]
number_templates = []

for num in range(10):
    for i in range(90):
        tmp = np.zeros(template_size)
        df = dfs[num][i]
        templates[num] += rasterize_number(df, SIZE)

    templates[num] /= np.sum(templates[num])

    image = np.array(templates[num])

    scaled_image = ((image - np.min(image)) / (np.max(image) - np.min(image))) * 255
    scaled_image[image == 0] = 0

    # plt.imshow(scaled_image, cmap='gray')
    # plt.show()


    thresh = cv2.threshold(scaled_image, 127, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.flip(cv2.transpose(thresh), 1)
    # plt.imshow(thresh, cmap='gray')
    # plt.show()
    #skeleton = (255*skeletonize(zero)).astype(np.uint8)
    skeleton = skeletonize(thresh)

    # plt.imshow(skeleton, cmap='gray')
    # plt.title('Skeletonized Image')
    # plt.show()

    coordinates = np.column_stack(np.where(skeleton == True))
    #print(coordinates)
    # print(coordinates.T[0])
    # print(coordinates[1])
    df = pd.DataFrame(coordinates, columns=['X', 'Y'])
    #df["Z"] = 0
    df = (df - df.mean()) / df.std()
    number_templates.append(df)

In [98]:
# plt.scatter(number_templates[2]['X'],number_templates[2]['Y'] , cmap='gray')
# plt.show()

# templates = [dfs[0][9], dfs[1][86], dfs[2][10], dfs[3][8], dfs[4][11], dfs[5][4], dfs[6][2], dfs[7][9], dfs[8][14], dfs[9][0]]
# plt.scatter(templates[9][0],templates[9][1] , cmap='gray')
# plt.show()

In [117]:
def hausdorff_distance(matrix1, matrix2):
    distance_matrix = cdist(matrix1, matrix2, metric='euclidean')
    max_dist_1_to_2 = np.max(np.min(distance_matrix, axis=1))
    max_dist_2_to_1 = np.max(np.min(distance_matrix, axis=0))
    return max(max_dist_1_to_2, max_dist_2_to_1)

def sum_of_min_distances(set1, set2):
    tree = KDTree(set1)
    distances = tree.query(set2)[0]
    normalized_sum = sum(distances) / len(set2)
    return normalized_sum

def calculate_metric(set1, set2):
    return sum_of_min_distances(set1, set2)
    return hausdorff_distance(set1, set2)

In [118]:
DEBUG=False
templates = number_templates

test_number = 0
correct = 0
all = 0
for df in dfs:
    for i in range(100):
        sample = df[i]
        minimum = 0
        pred = 0
        index = 0
        for template in templates:
            metric = calculate_metric(sample.values, template.values)
            if minimum > metric or index == 0:
                pred = index
                minimum = metric
            index += 1
        all += 1
        if pred == test_number:
            correct +=1
        elif DEBUG:
            print(f'Correct {test_number}, guess {pred}')
    test_number +=1

print(correct/all * 100)

37.7
